In [2]:
from tensorflow import keras

from tensorflow.keras import models 
from tensorflow.keras import layers 
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 

In [3]:
pd.set_option('mode.chained_assignment', None)

# Preparing the dataset and creating the features and target arrays


n_sims = 10000
n_list = [i for i in range(n_sims)]


h0s = []
results = []


start = 0
end = 10000

x = []
y =[]

for i in n_list[start:end]:
	
	data = np.load(f'input/data_{i+1}.npy')
	x.append(data[:, :2])
	y.append(data[:, 2:][0][0])
	print(f'Status: {((i+1)/end * 100):.2f} %', end='\r')


x = np.array(x)
# x[:,:,0] /= x[:,:,0].max()

y = np.array(y)



kfold = KFold(n_splits=5, shuffle=True)


In [4]:
# K-fold Cross Validation

fold_no = 1

mae_per_fold = []
mse_per_fold = []
r2_per_fold = []
rmse_per_fold = []


for train, test in kfold.split(x, y):

    model = models.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(16))
    model.add(layers.Dense(16))
    model.add(layers.Dense(1))
    model.compile(loss=keras.losses.MeanSquaredError(), optimizer='adam', metrics=keras.metrics.MeanAbsoluteError())


    print('\n------------------------------------------------------------------------\n')
    print(f'Training for fold {fold_no} ...\n')


    history = model.fit(x[train], y[train], epochs=25, batch_size=8, verbose=0)


    # scores = model.evaluate(x[test], y[test], verbose=0)

    y_pred = model.predict(x[test], verbose=0)


    r2 = r2_score(y[test], y_pred)
    mse = mean_squared_error(y[test], y_pred)
    mae = mean_absolute_error(y[test], y_pred)
    rmse = root_mean_squared_error(y[test], y_pred)


    print(f'Score for fold {fold_no}: \nMAE = {mae:.3f}\nR2 = {r2:.4f}\nMSE = {mse:.3f}\nRMSE = {rmse:.3f}')



    mae_per_fold.append(mae)
    mse_per_fold.append(mse)
    r2_per_fold.append(r2)
    rmse_per_fold.append(rmse)

    # Increase fold number
    fold_no += 1


------------------------------------------------------------------------


Training for fold 1 ...

Score for fold 1: 
MAE = 0.066
R2 = 0.9997
MSE = 0.006
RMSE = 0.077

------------------------------------------------------------------------


Training for fold 2 ...

Score for fold 2: 
MAE = 0.033
R2 = 0.9999
MSE = 0.002
RMSE = 0.039

------------------------------------------------------------------------


Training for fold 3 ...

Score for fold 3: 
MAE = 0.063
R2 = 0.9997
MSE = 0.005
RMSE = 0.071

------------------------------------------------------------------------


Training for fold 4 ...

Score for fold 4: 
MAE = 0.192
R2 = 0.9979
MSE = 0.040
RMSE = 0.199

------------------------------------------------------------------------


Training for fold 5 ...

Score for fold 5: 
MAE = 0.181
R2 = 0.9981
MSE = 0.034
RMSE = 0.184


In [5]:
print(f'MAE: {np.mean(mae_per_fold):.4f}\n')

print(f'MSE: {np.mean(mse_per_fold):.4f}\n')

print(f'R2: {np.mean(r2_per_fold):.4f}\n')

print(f'RMSE: {np.mean(rmse_per_fold):.4f}\n')

MAE: 0.1071

MSE: 0.0172

R2: 0.9991

RMSE: 0.1140

